In [70]:
import ipyleaflet as lf
from importlib import reload
import shapely.geometry as sg
import datetime as dt
from collections import defaultdict
import time

In [71]:
import busboy.map.map as mp
import busboy.constants as c
import busboy.util as u
import busboy.prediction as p
import busboy.database as db
import busboy.model as m
import busboy.apis as api

reload(mp)
reload(c)
reload(u)
reload(p)
reload(db)
reload(m)
reload(api)

<module 'busboy.apis' from '/Users/Noel/Developer/Projects/Busboy/busboy/apis.py'>

In [36]:
themap = mp.Map()
themap.map

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [41]:
themap.clear_layers()

In [5]:
all_stops = db.stops()
stops_by_name = {s.name: s for s in all_stops}

In [6]:
timetables_220 = list(api.timetables("220", stops_by_name))

In [65]:
len(list(u.unique(v for t in timetables_220 for v in t.variants if v.route == "220")))

10

In [72]:
timetables_205 = list(api.timetables("205", stops_by_name))

In [7]:
timetable = u.first(timetables_220)
variant = u.first(timetable.optional().variants)
variant.bind(lambda v: u.first(v.stops))

Just(value=Stop(id=StopId(raw='7338653551722184832'), name='Ovens (Grange Road Terminus)', latitude=51.87648, longitude=-8.64687, number=246671))

In [73]:
for v in u.unique(v for t in timetables_205 for v in t.variants if v.route == "205"):
    plot(v.stops, 0.1)

In [57]:
t = timetables_220[-2]
plot(list(t.variants)[0].stops, 0.1)

In [55]:
def plot(stops, interval):
    time.sleep(4)
    themap.clear_layers()
    for s in stops:
        themap._add_marker(s.latitude, s.longitude, s.name)
        time.sleep(interval)

In [67]:
stops = c.stops_on_220

In [5]:
for s in stops:
    themap.map.add_layer(
        lf.Marker(
                location=s.lat_lon,
                draggable=False,
                title=s.name,
            )
    )

In [9]:
for r in rs:
    themap.map.remove_layer(r)

AttributeError: 'Polygon' object has no attribute 'model_id'

In [68]:
rs = p.route_sections(stops, 0.001)

for r in rs:
    themap.add_polygon(r.polygon)

In [44]:
entries = db.entries(r = db.routes_by_name()["220"].id, d = dt.date(2019, 1, 5))
entries_by_trip = defaultdict(list)
for e in entries:
    entries_by_trip[e.trip].append(e)

In [56]:
list(u.take(10, ((t, len(v)) for t, v in entries_by_trip.items())))

[(TripId(raw='7338656568274099974'), 161),
 (TripId(raw='7338656568301104900'), 127),
 (TripId(raw='7338656568274099973'), 201),
 (TripId(raw='7338656568260779780'), 142),
 (TripId(raw='7338656568274099972'), 165),
 (TripId(raw='7338656568274185987'), 214),
 (TripId(raw='7338656568294059783'), 152),
 (TripId(raw='7338656568301104902'), 212),
 (TripId(raw='7338656568294059782'), 157),
 (TripId(raw='7338656568301104903'), 275)]

In [57]:
demo_trip = m.TripId(raw='7338656568274099974')
demo_entries = sorted(entries_by_trip[demo_trip], key=lambda e: e.last_modified)
sections = route_sections(stops)
demo_regions = p.assign_regions(demo_entries, stops)
demo_stops = list(p.most_recent_stops(demo_regions))

In [58]:
[(t[0].last_modified.isoformat(), t[1].map(lambda s: s.name).optional()) for t in demo_stops]

[('2019-01-05T10:48:59.888000', 'Carrigaline (Town Ctr Bridge Southbound)'),
 ('2019-01-05T11:23:52.049000', 'Douglas Road (Clermont Ave)'),
 ('2019-01-05T11:24:12.110000', 'Douglas Road (Clermont Ave)'),
 ('2019-01-05T11:24:28.163000', 'Douglas Road (Clermont Ave)'),
 ('2019-01-05T11:24:45.230000', 'Douglas Road (Clermont Ave)'),
 ('2019-01-05T11:24:59.255000', 'Douglas Road (Clermont Ave)'),
 ('2019-01-05T11:25:15.307000', 'Douglas Road (Clermont Ave)'),
 ('2019-01-05T11:25:19.909000', 'Douglas Road (Clermont Ave)'),
 ('2019-01-05T11:25:39.394000', 'Douglas Road (Clermont Ave)'),
 ('2019-01-05T11:26:10.486000', 'Douglas Road (Clermont Ave)'),
 ('2019-01-05T11:26:26.538000', 'Douglas Road (Clermont Ave)'),
 ('2019-01-05T11:26:26.554000', 'Douglas Road (Clermont Ave)'),
 ('2019-01-05T11:26:42.591000', 'Douglas Road (Clermont Ave)'),
 ('2019-01-05T11:27:09.673000', 'Douglas Road (Clermont Ave)'),
 ('2019-01-05T11:27:15.695000', 'Douglas Road (Clermont Ave)'),
 ('2019-01-05T11:27:29.7350